In [ ]:
#This code is complementary to Axion_PTall_EB-(hf)NB-detection.ipynb
#It helps to decide the values of hfNB detction (hfNB_max_duration and hfNB_max_time_interval)

In [ ]:
#Import libraries
library(reshape2)
library(stringr)
library(tidyverse)
library(dplyr)
library(data.table)
library(ggbiplot)

In [ ]:
#Insert working directory where plots for NB and hfNB discrimination can be saved.
setwd("insert path") 

#Path to main folder containing subfolders with analysis results from Axion_PTall_EB-(hf)NB-detection_Github-version.ipynb
parent.folder<-"insert_parent_folder"
#Search for all csv's that are called 'nb_times_s', sigma, '.csv' 
#Change the sigma to the selected sigma from Axion_PTall_EB-(hf)NB-detection_Github-version.ipynb
all_summary_files <- list.files(parent.folder, full.names = T, recursive = T, pattern = "nb_times_s120.csv")

all_files_dir<-dirname(all_summary_files)
#Check that only the NB times of analysis without hfNBdetection are selected
all_summary_files_nohf_axion<-str_subset(all_summary_files, "hfNBdetectionOFF")
length(all_summary_files_nohf_axion)
all_summary_files_nohf_axion
length(all_summary_files)

In [ ]:
#Remove any hfNB times from the list
all_summary_files <-all_summary_files[all_summary_files %in% all_summary_files_nohf_axion]
all_summary_files_remove<-str_subset(all_summary_files, "/hf_")
all_summary_files <-all_summary_files[!all_summary_files %in% all_summary_files_remove]

all_summary_files <- lapply(all_summary_files,read.csv, fileEncoding="UTF-8-BOM")
all_summary_files<-do.call(rbind, all_summary_files)
all_summary_files<-all_summary_files[-1]

#Construct a cell line overview containing at least the following columns:
#'Syndrome', 'Gene', 'Cell_line_name', 'Identity', 'Own_control', 'isogenic', 'Pool'


cell_line_overview<-read.csv("insert_path/Cell_line_overview.csv", fileEncoding="UTF-8-BOM")
full_results<-dplyr::left_join(all_summary_files, cell_line_overview, by=c("Base_Phenotype" = "Cell_line_name"))
#Check the dataframe
head(all_summary_files)

In [ ]:
#Remove any rows in which the 'Phenotype' is 'Exclude'
exclude_cell_lines<-rownames(full_results[full_results$Phenotype=="Exclude",])
full_results<-full_results[!rownames(full_results) %in% exclude_cell_lines,]

In [ ]:
#Create unique IDs for each well for each plate.
n_ID<-c(1:dim(full_results)[1])
full_results$Unique_ID<-paste0(full_results$PT_all_path,"_",full_results$Well_Label,"_",n_ID)
rownames(full_results)<-full_results$Unique_ID

In [ ]:
#Compute the minimum time intervals of NBs by taking the minimum of back and forward time intervals per NB
NB_time_interval_back<-full_results$NB_time_interval_back
NB_time_interval_forward<-full_results$NB_time_interval_forward
full_results$short_NB_time_interval<-pmin(NB_time_interval_back, NB_time_interval_forward,na.rm = TRUE)#na.rm, if there is na, pick the other value
full_results<-full_results %>% drop_na(c("mean_firing_rate_in_nb","duration","short_NB_time_interval"))#remove NAs

#Write these results to a csv
write.csv(full_results,"All_NB_times_s120_prop40_NB9hz25_final.csv")
head(full_results)
any(is.na(full_results$short_NB_time_interval))

In [ ]:
#Now plot all NB time intervals in a histogram. This will help to decide what the valuse of hfNB_max_time_interval should be.
p<-ggplot(full_results, aes(x=short_NB_time_interval)) +   
  
   geom_density(color="white", fill="red") + theme_minimal() +scale_x_continuous(limits = c(-0, 3),
                                                                                 breaks = seq(0, 3, by = 0.1))+
    theme(axis.text.x = element_text(angle = 45, hjust = 1))

tiff(paste0("short_NB_time_interval.tiff"), res =400, units = "in", width = 6, height = 2)
p
dev.off()
p

In [ ]:
#Plot all NB durations in a histogram. This will help to decide what the valuse of hfNB_max_duration should be.
p<-ggplot(full_results, aes(x=duration)) +   
  
   geom_density(color="white", fill="red") + theme_minimal() +scale_x_continuous(limits = c(-0, 1)) 
tiff(paste0("duration.tiff"), res =400, units = "in", width = 6, height = 2)
p
dev.off()
p